In [11]:
!pip3 install pyspark


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("KaggleParticipation").config(
    "spark.driver.memory", "4g"
).config("spark.executor.memory", "4g").getOrCreate()


25/05/19 16:08:37 WARN Utils: Your hostname, LT-W-7826.local resolves to a loopback address: 127.0.0.1; using 10.53.152.26 instead (on interface en0)
25/05/19 16:08:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/19 16:08:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
users = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/users.csv",
    header=True,
    inferSchema=True,
)
competitions = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv",
    header=True,
    inferSchema=True,
)
teams = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/teams.csv",
    header=True,
    inferSchema=True,
)
submissions = spark.read.csv(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/submissions.csv",
    header=True,
    inferSchema=True,
)


In [5]:
for df, name in [
    (users, "users"),
    (competitions, "competitions"),
    (teams, "teams"),
    (submissions, "submissions"),
]:
    print(f"—— schema of {name} ——")
    df.printSchema()


—— schema of users ——
root
 |-- Id: integer (nullable = true)
 |-- UserName: string (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- RegisterDate: string (nullable = true)
 |-- PerformanceTier: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationSharingOptOut: string (nullable = true)

—— schema of competitions ——
root
 |-- Id: string (nullable = true)
 |-- Slug: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- HostSegmentTitle: string (nullable = true)
 |-- ForumId: string (nullable = true)
 |-- OrganizationId: string (nullable = true)
 |-- EnabledDate: string (nullable = true)
 |-- DeadlineDate: string (nullable = true)
 |-- ProhibitNewEntrantsDeadlineDate: string (nullable = true)
 |-- TeamMergerDeadlineDate: string (nullable = true)
 |-- TeamModelDeadlineDate: string (nullable = true)
 |-- ModelSubmissionDeadlineDate: string (nullable = true)
 |-- FinalLeaderboardHasBeenVerified: str

In [3]:
submissions_df = submissions.alias("s")
teams_df = teams.alias("t")
users_df = users.alias("u")


In [10]:
from pyspark.sql.functions import col, to_date, regexp_replace, trim


clean_comp = (
    spark.read.option("header", True)
    .option("multiLine", True)
    .option("escape", '"')
    .csv(
        "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/data/competitions.csv"
    )
)

clean_comp = clean_comp.withColumn(
    "SlugClean", regexp_replace(col("Slug"), "<[^>]*>", "")
)
clean_comp = clean_comp.withColumn("SlugClean", trim(col("SlugClean")))

clean_comp_filtered = (
    clean_comp.filter(col("Id").rlike("^[0-9]+$"))
    .withColumn("CompetitionId", col("Id"))
    .withColumn("DeadlineDate", to_date("DeadlineDate", "yyyy-MM-dd"))
    .select("CompetitionId", "SlugClean", "DeadlineDate")
    .dropDuplicates(["CompetitionId"])
    .alias("c")
)

clean_comp_filtered.show(20, truncate=False)


+-------------+----------------------------------------------------------------------+------------+
|CompetitionId|SlugClean                                                             |DeadlineDate|
+-------------+----------------------------------------------------------------------+------------+
|10001        |method-pro-competiton                                                 |NULL        |
|100023       |2025-ness-statathon                                                   |NULL        |
|100032       |boost-global-music-genre-classification                               |NULL        |
|100045       |sampling-assisted-pathloss-rm-prediction-t-2-ii                       |NULL        |
|10006        |method-house-price-prediction                                         |NULL        |
|100070       |data-bounty-1-stock-forcasting                                        |NULL        |
|100073       |a-i-2025-02                                                           |NULL        |


In [12]:
sub_with_team = submissions.alias("s").join(
    teams.alias("t"), col("s.TeamId") == col("t.Id"), "left"
)

sub_with_team_comp = sub_with_team.join(
    clean_comp_filtered.alias("c"),
    col("t.CompetitionId") == col("c.CompetitionId"),
    "left",
)

sub_with_user = sub_with_team_comp.join(
    users.alias("u"), col("s.SubmittedUserId") == col("u.Id"), "left"
)

sub_with_user.select(
    col("s.Id").alias("SubmissionId"),
    col("u.UserName"),
    col("t.TeamName"),
    col("c.SlugClean").alias("Slug"),
    col("s.SubmissionDate"),
).show(5, truncate=False)


+------------+--------------+------------+-----------------------------------+--------------+
|SubmissionId|UserName      |TeamName    |Slug                               |SubmissionDate|
+------------+--------------+------------+-----------------------------------+--------------+
|35465835    |jcole75       |Jack Cole   |abstraction-and-reasoning-challenge|12/19/2023    |
|13652403    |katielicmu    |BeepBoop    |11785-fall19-hw4p2                 |12/06/2019    |
|22403697    |clairewumel   |Yan Shi     |tabular-playground-series-aug-2021 |08/17/2021    |
|31821410    |insuperabile  |insuperabile|icr-identify-age-related-conditions|05/17/2023    |
|39081448    |marianadeem755|maria nadeem|playground-series-s4e7             |07/09/2024    |
+------------+--------------+------------+-----------------------------------+--------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

competition_dim = (
    clean_comp_filtered
    .withColumn("CompetitionKey", row_number().over(Window.orderBy("CompetitionId")))
    .alias("cd")
)

competition_dim.printSchema()
competition_dim.show(5, truncate=False)


root
 |-- CompetitionId: string (nullable = true)
 |-- SlugClean: string (nullable = true)
 |-- DeadlineDate: date (nullable = true)
 |-- CompetitionKey: integer (nullable = false)



25/05/19 16:17:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:17:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:17:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:17:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:17:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:17:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+-----------------------------------------------+------------+--------------+
|CompetitionId|SlugClean                                      |DeadlineDate|CompetitionKey|
+-------------+-----------------------------------------------+------------+--------------+
|10001        |method-pro-competiton                          |NULL        |1             |
|100023       |2025-ness-statathon                            |NULL        |2             |
|100032       |boost-global-music-genre-classification        |NULL        |3             |
|100045       |sampling-assisted-pathloss-rm-prediction-t-2-ii|NULL        |4             |
|10006        |method-house-price-prediction                  |NULL        |5             |
+-------------+-----------------------------------------------+------------+--------------+
only showing top 5 rows



In [15]:
from pyspark.sql.functions import dayofmonth, month, year, hour, date_format

user_dim = (
    users.select("Id", "UserName", "Country", "PerformanceTier")
    .dropDuplicates()
    .withColumn("UserKey", row_number().over(Window.orderBy("Id")))
    .alias("ud")
)
user_dim.show(5, truncate=False)

team_dim = (
    teams.select("Id", "TeamName", "IsBenchmark")
    .dropDuplicates()
    .withColumn("TeamKey", row_number().over(Window.orderBy("Id")))
    .alias("td")
)
team_dim.show(5, truncate=False)

time_dim = (
    submissions.alias("s")
    .select("SubmissionDate")
    .dropDuplicates()
    .withColumn("TimeKey", to_date("SubmissionDate", "MM/dd/yyyy"))
    .filter(col("TimeKey").isNotNull())
    .withColumn("Day", dayofmonth("TimeKey"))
    .withColumn("Month", month("TimeKey"))
    .withColumn("Year", year("TimeKey"))
    .withColumn("Hour", hour("SubmissionDate"))
    .withColumn("DayOfWeek", date_format("TimeKey", "EEEE"))
    .select("TimeKey", "Day", "Month", "Year", "Hour", "DayOfWeek")
    .alias("tdim")
)
time_dim.show(5, truncate=False)


25/05/19 16:18:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:18:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:18:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:18:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:18:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:18:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:18:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:18:46 WARN RowBasedKeyVal

+---+------------+-------------+---------------+-------+
|Id |UserName    |Country      |PerformanceTier|UserKey|
+---+------------+-------------+---------------+-------+
|1  |kaggleteam  |NULL         |5              |1      |
|368|antgoldbloom|United States|2              |2      |
|381|iguyon      |United States|2              |3      |
|383|davidstephan|Australia    |0              |4      |
|384|gabewarren  |Australia    |0              |5      |
+---+------------+-------------+---------------+-------+
only showing top 5 rows



25/05/19 16:19:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:19:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:19:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:19:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:19:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:19:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------------+-----------+-------+
|Id |TeamName    |IsBenchmark|TeamKey|
+---+------------+-----------+-------+
|496|team1       |False      |1      |
|497|jonp        |False      |2      |
|499|Bwaas       |False      |3      |
|500|Thylacoleo  |False      |4      |
|501|pjonesdotcda|False      |5      |
+---+------------+-----------+-------+
only showing top 5 rows



+----------+---+-----+----+----+---------+
|TimeKey   |Day|Month|Year|Hour|DayOfWeek|
+----------+---+-----+----+----+---------+
|2011-03-02|2  |3    |2011|NULL|Wednesday|
|2012-01-03|3  |1    |2012|NULL|Tuesday  |
|2012-05-19|19 |5    |2012|NULL|Saturday |
|2012-12-01|1  |12   |2012|NULL|Saturday |
|2013-11-17|17 |11   |2013|NULL|Sunday   |
+----------+---+-----+----+----+---------+
only showing top 5 rows



In [17]:
facts = (
    sub_with_user.join(user_dim, col("s.SubmittedUserId") == col("ud.Id"), "left")
    .join(team_dim, col("s.TeamId") == col("td.Id"), "left")
    .join(competition_dim, col("t.CompetitionId") == col("cd.CompetitionId"), "left")
    .join(
        time_dim,
        to_date(col("s.SubmissionDate"), "MM/dd/yyyy") == col("tdim.TimeKey"),
        "left",
    )
    .select(
        col("s.Id").alias("SubmissionId"),
        col("tdim.TimeKey"),
        col("ud.UserKey"),
        col("td.TeamKey"),
        col("cd.CompetitionKey"),
        col("s.PublicScoreLeaderboardDisplay"),
        col("s.PrivateScoreLeaderboardDisplay"),
        col("s.IsAfterDeadline"),
    )
    .repartition(10)
)


In [18]:
user_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/final/UserDim"
)
team_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/final/TeamDim"
)
competition_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/final/CompetitionDim"
)
time_dim.write.json(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/final/TimeDim"
)
facts.write.mode("overwrite").parquet(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/final/SubmissionFact"
)


25/05/19 16:20:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:20:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:20:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:20:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:20:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:20:41 WARN RowBasedKeyVal

In [19]:
df = spark.read.parquet(
    "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/final/SubmissionFact/part-00000-39e907ca-3957-457c-acda-c96fba993cb0-c000.snappy.parquet"
)
df.show()
df.printSchema()


+------------+----------+--------+-------+--------------+-----------------------------+------------------------------+---------------+
|SubmissionId|   TimeKey| UserKey|TeamKey|CompetitionKey|PublicScoreLeaderboardDisplay|PrivateScoreLeaderboardDisplay|IsAfterDeadline|
+------------+----------+--------+-------+--------------+-----------------------------+------------------------------+---------------+
|    25826845|2022-04-02| 2567524|5157860|          4917|                      67.2359|                      66.67882|          false|
|    21137915|2021-05-18| 6280305|4184374|          3393|                      0.73464|                       0.72509|          false|
|     8112860|2018-08-08|  905845|1178984|            96|                      0.74774|                       0.77301|          false|
|    38317113|2024-05-26| 4519586|5367910|          5569|                      0.57081|                       0.59696|           true|
|     7931466|2018-07-12| 1307599|1123308|          938

In [ ]:
# user_dim = spark.read.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/UserDim"
# )

# team_dim = spark.read.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TeamDim"
# )

# competition_dim = spark.read.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/CompetitionDim"
# )

# time_dim = spark.read.json(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/TimeDim"
# )

# facts = spark.read.parquet(
#     "/Users/denys.koval/Labs/projects/kaggle-participation-analysis/output/SubmissionFact"
# )


## User Dim
Metric: Average Public Score per User

In [24]:
import pyspark.sql.functions as F

agg_user = (
    facts.groupBy("UserKey")
    .agg(F.avg("PublicScoreLeaderboardDisplay").alias("AvgPublicScore"))
    .filter(F.col("AvgPublicScore") < 1)
)
agg_user.show()


25/05/19 16:29:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:29:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:29:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:29:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:29:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:29:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 1

+--------+-------------------+
| UserKey|     AvgPublicScore|
+--------+-------------------+
| 2248728| 0.5282915384615385|
| 3620091|0.26163821428571427|
| 5888405| 0.5269999166666666|
| 1808191| 0.6546666666666666|
|  325894|0.37472357142857143|
| 7980038|  0.783136931818182|
| 1641148| 0.6689276756756757|
| 5442355| 0.7885235802469136|
|14752065|            0.72336|
|  196628| 0.7259273684210527|
|  449275| 0.4312530263157895|
| 2797037|  0.600550752688172|
| 2902540| 0.8896227272727273|
| 3510210|              0.933|
|   75149| 0.7814723611111111|
|  367222| 0.6913452427184467|
| 1572810|            0.40303|
|13212659| 0.8085695454545455|
|  485546| 0.5330961538461538|
| 4339808|           0.665953|
+--------+-------------------+
only showing top 20 rows



## Team Dim
Metric: Total number of submissions per Team

In [21]:
agg_team = facts.groupBy("TeamKey").agg(F.count("*").alias("SubmissionCount"))
agg_team.show()


25/05/19 16:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:26:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:26:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:26:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:26:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:26:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 1

+-------+---------------+
|TeamKey|SubmissionCount|
+-------+---------------+
|7348020|              4|
| 172959|             28|
| 650300|             12|
| 108560|              7|
|4067199|             10|
|1901931|             10|
|2442917|            130|
| 217657|             10|
| 228725|             14|
| 602759|             13|
| 102594|              9|
|6961324|              2|
|4367868|              8|
|4235032|             14|
|  22223|              6|
|4073153|              2|
|8008295|              4|
|7423721|             12|
|4009338|              8|
|4489935|             10|
+-------+---------------+
only showing top 20 rows



## Competition Dim
Metric: Average private score per competition

In [26]:
agg_comp = (
    facts.groupBy("CompetitionKey")
    .agg(
        F.avg("PrivateScoreLeaderboardDisplay")
        .alias("AvgPrivateScore")
    )
    .filter(F.col("AvgPrivateScore") < 1)
)
agg_comp.show()


25/05/19 16:32:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:32:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:32:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:32:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:32:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 16:32:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/05/19 1

+--------------+-------------------+
|CompetitionKey|    AvgPrivateScore|
+--------------+-------------------+
|          6658|0.04110812937619347|
|          6357|  0.601969388707419|
|          3794| 0.6996558513189446|
|          7554| 0.4327419026149684|
|          2866|0.45826998223801063|
|          1959| 0.6420320855614974|
|          1829| 0.6517253482260184|
|          7754| 0.6929542857142857|
|          4935| 0.7700231779661015|
|          7993| 0.6735545272206304|
|           496| 0.6894351505791508|
|          1088| 0.8992568068535824|
|           148| 0.9341468789808919|
|          4101| 0.6924480314960628|
|          2659| 0.4387096774193549|
|          1238| 0.7562620689655174|
|          3175| 0.8552204285714287|
|           833|           0.859824|
|          1591| 0.6289066666666666|
|          4818|  0.700068275862069|
+--------------+-------------------+
only showing top 20 rows



## Time Dim
Metric: Number of submissions per year-month

In [23]:
from pyspark.sql.functions import year, month

agg_time = (
    facts.withColumn("Year", year("TimeKey"))
    .withColumn("Month", month("TimeKey"))
    .groupBy("Year", "Month")
    .agg(F.count("*").alias("SubmissionCount"))
    .orderBy("Year", "Month")
)
agg_time.show()


25/05/19 16:28:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/19 16:28:58 WARN RowBasedKeyValueBatch: Calling spill() on

+----+-----+---------------+
|Year|Month|SubmissionCount|
+----+-----+---------------+
|2010|    4|              3|
|2010|    5|            186|
|2010|    6|            304|
|2010|    7|            612|
|2010|    8|           1528|
|2010|    9|           1703|
|2010|   10|           1925|
|2010|   11|           1487|
|2010|   12|           2263|
|2011|    1|           3157|
|2011|    2|           3708|
|2011|    3|           2551|
|2011|    4|           4294|
|2011|    5|           2616|
|2011|    6|           1682|
|2011|    7|           2258|
|2011|    8|           3445|
|2011|    9|           3607|
|2011|   10|           5116|
|2011|   11|           7653|
+----+-----+---------------+
only showing top 20 rows

